In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dataframe = pd.read_csv('data.csv')
dataframe.head()

index    area  bathrooms     price    sq_price
0      0  2104.0        3.0  399900.0  190.066540
1      1  1600.0        3.0  329900.0  206.187500
2      2  2400.0        3.0  369000.0  153.750000
3      3  1416.0        2.0  232000.0  163.841808
4      4  3000.0        4.0  539900.0  179.966667

In [4]:
dataframe = dataframe.drop(['index', 'price', 'sq_price'], axis=1)
dataframe = dataframe[0:10]
dataframe

area  bathrooms
0  2104.0        3.0
1  1600.0        3.0
2  2400.0        3.0
3  1416.0        2.0
4  3000.0        4.0
5  1985.0        4.0
6  1534.0        3.0
7  1427.0        3.0
8  1380.0        3.0
9  1494.0        3.0

In [5]:
# Add Labels, 1 := good buy, 0 := poor buy
dataframe.loc[:, ('y1')] = [1, 1, 1, 0, 0, 1, 0, 1, 1, 1]
dataframe.loc[:, ('y2')] = dataframe['y1'] == 0
dataframe.loc[:, ('y2')] = dataframe['y2'].astype(int)

dataframe

area  bathrooms  y1  y2
0  2104.0        3.0   1   0
1  1600.0        3.0   1   0
2  2400.0        3.0   1   0
3  1416.0        2.0   0   1
4  3000.0        4.0   0   1
5  1985.0        4.0   1   0
6  1534.0        3.0   0   1
7  1427.0        3.0   1   0
8  1380.0        3.0   1   0
9  1494.0        3.0   1   0

In [6]:
# Prepare Data for TF

# Convert features and labels to two separate 2D (input) tensors
inputX = dataframe.loc[:, ['area', 'bathrooms']].as_matrix()
inputY = dataframe.loc[:, ['y1', 'y2']].as_matrix()
inputX

array([[  2.10400000e+03,   3.00000000e+00],
       [  1.60000000e+03,   3.00000000e+00],
       [  2.40000000e+03,   3.00000000e+00],
       [  1.41600000e+03,   2.00000000e+00],
       [  3.00000000e+03,   4.00000000e+00],
       [  1.98500000e+03,   4.00000000e+00],
       [  1.53400000e+03,   3.00000000e+00],
       [  1.42700000e+03,   3.00000000e+00],
       [  1.38000000e+03,   3.00000000e+00],
       [  1.49400000e+03,   3.00000000e+00]])

In [7]:
inputY

array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0]])

In [21]:
# Set hyperparameters
learning_rate = 0.000001
training_epochs = 2000
sample_size = inputY.size

In [22]:
# Create  TF computation graph (neural network)

# Feature tensor
x = tf.placeholder(tf.float32, [None, 2])

# Weights
W = tf.Variable(tf.zeros([2, 2]))

# biases
b = tf.Variable(tf.zeros([2]))

# Multiply the weights by the inputs and add the biases
y_values = tf.add(tf.matmul(x, W), b)

# Apply our activation function (softmax = sigmoid)
y = tf.nn.softmax(y_values)

# Feed in the labels
y_ = tf.placeholder(tf.float32, [None, 2])

In [23]:
# Cost functione, here mean squard error
cost = tf.reduce_sum(tf.pow(y_ - y, 2)) / (2*sample_size)

# Optimization via gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [24]:
# Initialize variables
init = tf.initialize_all_variables()
session = tf.Session()
session.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [25]:
# Train NN
for i in range(training_epochs):
    session.run(optimizer, feed_dict={x: inputX, y_: inputY})
    
    if (i) % 50 == 0:
        cc = session.run(cost, feed_dict={x: inputX, y_: inputY})
        print("Training step: ", '%04d' % (i), "cost=", "{:.9f}".format(cc))
        
print("\nOptimization finished")
trainging_cost = session.run(cost, feed_dict={x: inputX, y_: inputY})
print("Training cost=", trainging_cost)
print("W=", session.run(W))
print("b=", session.run(b))

Training step:  0000 cost= 0.114958666
Training step:  0050 cost= 0.109539941
Training step:  0100 cost= 0.109539881
Training step:  0150 cost= 0.109539807
Training step:  0200 cost= 0.109539732
Training step:  0250 cost= 0.109539673
Training step:  0300 cost= 0.109539606
Training step:  0350 cost= 0.109539531
Training step:  0400 cost= 0.109539464
Training step:  0450 cost= 0.109539405
Training step:  0500 cost= 0.109539330
Training step:  0550 cost= 0.109539248
Training step:  0600 cost= 0.109539196
Training step:  0650 cost= 0.109539129
Training step:  0700 cost= 0.109539054
Training step:  0750 cost= 0.109538987
Training step:  0800 cost= 0.109538913
Training step:  0850 cost= 0.109538853
Training step:  0900 cost= 0.109538779
Training step:  0950 cost= 0.109538712
Training step:  1000 cost= 0.109538652
Training step:  1050 cost= 0.109538577
Training step:  1100 cost= 0.109538510
Training step:  1150 cost= 0.109538436
Training step:  1200 cost= 0.109538376
Training step:  1250 cost

In [26]:
predictions = session.run(y, feed_dict={x: inputX})
predictions = pd.DataFrame(predictions).round()

diff = (predictions - pd.DataFrame(inputY)).astype(int)
accuracy = (diff[0].size - diff[0].sum()) / diff[0].size

print("accuracy: ", accuracy)
diff

accuracy:  0.7


0  1
0  0  0
1  0  0
2  0  0
3  1 -1
4  1 -1
5  0  0
6  1 -1
7  0  0
8  0  0
9  0  0